In [ ]:
!pip install transformers
!pip install -qqq bitsandbytes==0.39.0
!pip install peft trl datasets

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_dataset
import torch
from trl import SFTTrainer
import transformers
from accelerate import Accelerator

accelerator = Accelerator()
device = accelerator.device

# Preparing Dataset

In [3]:
fine_tuning_data=pd.read_csv(r"/content/midjourney training dataset - midjourney_prompt_dataset.csv")
fine_tuning_data.head()

,User,Prompt
0,"""midjourney prompt for a female character in a...","""< yoshida akihiko art, pixiv art, patreon art..."
1,"""midjourney prompt for a Viking warrior with a...","""Norse amulet made from black iron with a carb..."
2,midjourney prompt for a spooky spirit summonin...,1920s double-exposure spirit photograph of a b...
3,midjourney prompt for a steampunk children's c...,A blueprint of Steampunk style mini Children's...
4,"""midjourney prompt for a girl wearing roller s...","""A cute girl below the knee, sneakers"""


In [4]:
fine_tuning_data.shape

(289, 2)

In [5]:
for i in range(len(fine_tuning_data)):

    fine_tuning_data.loc[i,'Text']="### Instruction:"+str(fine_tuning_data.loc[i,'User'])+"### Response:"+str(fine_tuning_data.loc[i,'Prompt'])

fine_tuning_data.to_csv(r"/content/midjourney training dataset - midjourney_prompt_dataset-modified.csv",index=False)

In [6]:
fine_tuning_data=load_dataset('csv',data_files=r"/content/midjourney training dataset - midjourney_prompt_dataset-modified.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
fine_tuning_data

DatasetDict({
    train: Dataset({
        features: ['User', 'Prompt', 'Text'],
        num_rows: 289
    })
})

In [22]:
def processing_data(data_point):

  return tokenizer(data_point['Text'], padding=True, truncation=True)


fine_tuning_data=fine_tuning_data['train'].map(processing_data)
fine_tuning_data

Map:   0%|          | 0/289 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dataset({
    features: ['User', 'Prompt', 'Text', 'input_ids', 'attention_mask'],
    num_rows: 289
})

# Loading Quantized Model

In [8]:
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoModelForCausalLM

from peft import (
    LoraConfig,
    get_peft_model,
)

from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

In [18]:
MODEL_NAME = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    load_in_8bit_fp32_cpu_offload=True
)


model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

# data = torch.utils.data.DataLoader(fine_tuning_data, shuffle=True)

# model,  data = accelerator.prepare(model, data)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

# Fine-Tuning Model

In [24]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    # target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

max_seq_length = 512

training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)


trainer = SFTTrainer(
    model=model,
    train_dataset=fine_tuning_data,
    peft_config=config,
    dataset_text_field="Text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
)

Map:   0%|          | 0/289 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:207: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [25]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
1,3.945600
2,3.621700
3,3.596900
4,3.713700
5,3.454600
6,3.165300
7,3.382700
8,3.129000
9,2.772600
10,2.626300


TrainOutput(global_step=72, training_loss=2.0688763277398214, metrics={'train_runtime': 421.7372, 'train_samples_per_second': 0.685, 'train_steps_per_second': 0.171, 'total_flos': 482116847591424.0, 'train_loss': 2.0688763277398214, 'epoch': 1.0})

# Saving Model

In [26]:
trainer.save_model('/content/Fine_Tuning_Llama-2-7B')

# Inference

In [43]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto"
)

In [46]:
output=pipeline("### Instruction: midjourney prompt for a retro-futuristic spaceship with organic shapes and bright colors' ### Response:",max_new_tokens=150,repetition_penalty=2.0)

output[0]['generated_text']

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


"### Instruction: midjourney prompt for a retro-futuristic spaceship with organic shapes and bright colors' ### Response: Spaceships, futurespace artwork by HR Giger --ar 16 :9 ::unreal engine::organically shaped ships in the middle of space. nobody around but lotsa neon lights & glowing particles + black void backgrounds (cinematic) epapier soft light matte painting style highly detailed cinamatic realism hyperdetailed photorealist cgsociety renderwars conceptart stationery papercraft minimal design trending on ArtStation environment landscape abstract surreality uncanny valley dark moody moodysomber gloomy melancholicy depressive sadness loneliness isolation lonely desolate barren lifeless empty cold harsh hostile"